## Load Libraries

In [1]:
import os
import random
import wandb
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split


# wandb 프로젝트 설정
os.environ["WANDB_PROJECT"] = "Level2_data-centric-label-estimate"

## Set Hyperparameters

In [2]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [4]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

## Load Tokenizer and Model

In [5]:
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Dataset

In [6]:
data = pd.read_csv(os.path.join(DATA_DIR, 'self_train/self_train_v2_step1.csv'))
dataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=SEED)

In [ ]:
dataset_train

,ID,text,target
1040,ynat-v1_train_01040,에어로스페이스 아시아 인사이트 부인…분기 적자 축소,5
1599,ynat-v1_train_01599,민주일반연맹 비정규직 차별철폐 공동행동,3
194,ynat-v1_train_00194,서울자유시민대학 초대 학장에 정재권 전 한겨레신문 논설위원,3
2461,ynat-v1_train_02461,아베 개헌논의 안 해도 좋은지 선거서 물을 것…개헌 이슈화,6
1479,ynat-v1_train_01479,신간,0
...,...,...,...
2543,ynat-v1_train_02543,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,1
2090,ynat-v1_train_02090,성 베드로 대학 베트남 캠퍼스 개교…중국 가전 기업 동남아 공략,4
2649,ynat-v1_train_02649,아이팩토리 상장폐지 이의신청서 제출,5
613,ynat-v1_train_00613,LG전자 미국서 G6 사면 구글 홈 준다,4


In [8]:
dataset_valid

,ID,text,target
2711,ynat-v1_train_02711,신동빈 경영복귀 첫 일성 적극적 투자로 국가 경제 이바지종합,5
2156,ynat-v1_train_02156,나스닥 3개월만에 최저 폭락 종합,5
549,ynat-v1_train_00549,中공안 장쑤 화학공단 폭발참사 책임자들 구금,6
933,ynat-v1_train_00933,더위 식히는 장맛비…남부·제주도 밤에 대부분 그쳐,0
186,ynat-v1_train_00186,KB증권 농심 4분기 라면 부문 실적개선…목표주가↑,5
...,...,...,...
873,ynat-v1_train_00873,청소년 짙은 안경 착용 항공 회사의 운,0
1144,ynat-v1_train_01144,비례대표^^권마다^^복되는 논쟁...해법책^없나,3
1775,ynat-v1_train_01775,시즌 첫 골 손흥민 모든 상황 준비해 좋은 결과로 이어졌다,1
394,ynat-v1_train_00394,중국군 시리아 순찰…뒤이어,6


In [9]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', return_token_type_ids=False)
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

In [10]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [12]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


## Train Model

In [13]:
### for wandb setting
#os.environ['WANDB_DISABLED'] = 'true'

In [13]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR+"/self_train_ver2_step2",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    report_to="wandb",
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=50,
    save_steps=350,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jilp1598 (jilp1598-dd). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1
50,No log,0.929875,0.718641
100,4.365700,0.940283,0.714781
150,4.365700,0.868431,0.741656
200,2.232000,0.914859,0.741381
250,2.232000,0.963780,0.729756
300,1.180100,0.968096,0.733005
350,1.180100,0.978750,0.721874


TrainOutput(global_step=350, training_loss=2.3425853729248045, metrics={'train_runtime': 576.1866, 'train_samples_per_second': 19.438, 'train_steps_per_second': 0.607, 'total_flos': 1.04378076020736e+16, 'train_loss': 2.3425853729248045, 'epoch': 5.0})

In [13]:
labeled_data_path = os.path.join(DATA_DIR, "recovery_data")

# noise_list = []
# for label in range(7) :
#     df = pd.read_csv(labeled_data_path+f"/c1_label_dropped_{label}.csv")
#     noise_list.append(df)

# noise_df = pd.concat(noise_list, ignore_index=True)
df = pd.read_csv(labeled_data_path+f"/converted_train_ver2_fewshot.csv")
noise_df = df[df['is_noisy'] == 1]

In [14]:
ids = noise_df['ID'].values
ids

array(['ynat-v1_train_00000', 'ynat-v1_train_00001',
       'ynat-v1_train_00002', ..., 'ynat-v1_train_02790',
       'ynat-v1_train_02792', 'ynat-v1_train_02797'], dtype=object)

In [17]:
origin_train = pd.read_csv(os.path.join(DATA_DIR, 'self_train/self_train_v2_step1.csv'))

model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_DIR+"/self_train_ver2_step2/checkpoint-350").to(DEVICE)
model.eval()

new_train = origin_train.copy()

for idx, sample in tqdm(origin_train.iterrows(), total=len(origin_train), desc="Evaluating"):
    target_id = sample['ID']

    if target_id in ids :
        continue

    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        new_train.loc[new_train['ID'] == target_id, 'target'] = pred[0]

new_train.to_csv(f'../data/self_train/self_train_ver2_step2.csv', index=False)

Evaluating: 100%|██████████| 2800/2800 [00:10<00:00, 273.48it/s]


## Evaluate Model